In [1]:
import numpy as np

import pandas as pd

import scipy as sp
import scipy.interpolate

import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.figure_factory as ff

from itertools import combinations

from collections import OrderedDict


from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression

In [10]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import tadasets

In [6]:
data = np.random.randn(300, 3)
sigma = np.array([[2,0,0], [0,2,0], [0,0,1]])
R = [[np.cos(np.pi/4), 0, -np.sin(np.pi/4)], [0,1,0], [np.sin(np.pi/4), 0, np.cos(np.pi/4)]]

In [19]:
sample1 = data@sigma@R

In [101]:
sample2 = tadasets.torus(n=300, c = 4, noise = 0.3)@R

In [102]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scatter3d"},{"type": "scatter3d"}]])

fig.add_trace(
    go.Scatter3d(x=sample1[:,0], y=sample1[:,1], z=sample1[:,2],
                 mode='markers', marker_size=3, name='Sample 1'),
    row=1, col=1
)

model1 = LinearRegression()
model1.fit(sample1[:,[0,1]], sample1[:,[2]])
x_ols1 = np.linspace(sample1[:,[0,1]].min(), sample1[:,[0,1]].max(), 100)
y_ols1 = model1.predict(np.array([x_ols1,x_ols1]).T)
fig.add_trace(
    go.Scatter3d(x=x_ols1, y=x_ols1, z=y_ols1[:,0],
                 mode='lines', name='Fit 1'),
    row=1, col=1
)



fig.add_trace(
    go.Scatter3d(x=sample2[:,0], y=sample2[:,1], z=sample2[:,2],
                 mode='markers', marker_size=3, name='Sample 2'),
    row=1, col=2
)

model1 = LinearRegression()
model1.fit(sample2[:,[0,1]], sample2[:,[2]])
x_ols2 = np.linspace(sample1[:,[0,1]].min(), sample1[:,[0,1]].max(), 100)
y_ols2 = model1.predict(np.array([x_ols2,x_ols2]).T)
fig.add_trace(
    go.Scatter3d(x=x_ols2, y=x_ols2, z=y_ols2[:,0],
                 mode='lines', name='Fit 1'),
    row=1, col=2
)


# fig.update_xaxes(title_text="Sample 1", row=1, col=1)
# fig.update_xaxes(title_text="Sample 2", row=1, col=1)
fig.update_layout(height=500, width=1000, title_text="3D-Plots")

name = 'eye = (x:2.5, y:0., z:0.)'
camera = dict(
    eye=dict(x=0, y=2., z=0.)
)

fig.update_layout(scene_camera=camera,
                  scene2_camera=camera,
                 legend=dict(orientation='h'),
#     yanchor="bottom",
                 )#, title=name)
# fig.update_layout(scene2_camera=camera)
fig.show()

In [106]:
True == 1 == 1.0 == 1.0 +  0.0j

True